In [3]:
import pandas as pd
import matplotlib.pyplot as plt

import sklearn as skl
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

import nltk
from nltk.corpus import stopwords, wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

In [4]:
pd.set_option('max_colwidth', None)

In [5]:
# nltk.download('udhr')

In [6]:
# nltk.stem.snowball.demo()

In [45]:
course_data_df = pd.read_csv('Course Relevance Dataset.csv', encoding='latin-1')

course_data_df['Robust'] = course_data_df['Name Of the Program'] + ' ' + course_data_df['Type of Course'] + ' ' + \
course_data_df['Description of the need']

course_data_df.head()

,SrNo,Name Of the Program,Type of Course,Code,Need,Description of the need,Robust
0,1,BA (Geography),Environmental studies,UGEVS101,"Local, Regional, National and Global","To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment","BA (Geography) Environmental studies To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment"
1,2,BA (Geography),Environmental studies,UGEVS201,"Local, Regional, National and Global","Examining historical and current environmental movements provides insight into societal responses to environmental issues. Environmental management strategies help students understand how to balance development with conservation. Students will gain insights into waste disposal techniques, recycling processes, and the impact of improper waste management on the environment","BA (Geography) Environmental studies Examining historical and current environmental movements provides insight into societal responses to environmental issues. Environmental management strategies help students understand how to balance development with conservation. Students will gain insights into waste disposal techniques, recycling processes, and the impact of improper waste management on the environment"
2,3,BA (Geography),Environmental Science-I,UGESlOl,"Local, Regional, National and Global","To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment","BA (Geography) Environmental Science-I To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment"
3,4,BA (Geography),Environmental Science-II,UGES201,"Local, Regional, National and Global","Managing waste is critical for preventing environmental degradation. Students need to understand waste generation, treatment, and disposal methods to contribute to sustainable waste management practices.Students will comprehend the causes and effects of pollution, as well as the regulatory frameworks and policies aimed at environmental conservation.","BA (Geography) Environmental Science-II Managing waste is critical for preventing environmental degradation. Students need to understand waste generation, treatment, and disposal methods to contribute to sustainable waste management practices.Students will comprehend the causes and effects of pollution, as well as the regulatory frameworks and policies aimed at environmental conservation."
4,5,BA (Geography),Fundamentals of Geomorphology,UGGEO101,"Local, Regional, National and Global","These geological processes shape the Earth's surface. Understanding endogenic (internal) and exogenic (external) processes helps in comprehending landforms, earthquakes,\nand erosion.","BA (Geography) Fundamentals of Geomorphology These geological processes shape the Earth's surface. Understanding endogenic (internal) and exogenic (external) processes helps in comprehending landforms, earthquakes,\nand erosion."


In [46]:
row_nan_count = course_data_df.isnull().sum()
print(row_nan_count)

SrNo                       0
Name Of the Program        0
Type of Course             0
Code                       0
Need                       1
Description of the need    0
Robust                     0
dtype: int64


In [9]:
#  NLTK data - Only need to run once (I think).
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
# sample = course_data_df['Description of the need'][1]

In [11]:
## Source: https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python

# from collections import defaultdict

# lemmatizer = WordNetLemmatizer()

# tokens = word_tokenize(sample.lower())

# tag_map = defaultdict(lambda : wn.NOUN)
# tag_map['J'] = wn.ADJ
# tag_map['V'] = wn.VERB
# tag_map['R'] = wn.ADV

# l = []
# for token, tag in pos_tag(tokens):
#     lemma = lemmatizer.lemmatize(token, tag_map[tag[0]])
#     l.append(lemma)
# print(l)

In [48]:
# https://www.nltk.org/_modules/nltk/stem/wordnet.html
lemmatizer = WordNetLemmatizer()

# Function that processes the text input
def preprocess_text(text):
    #Tokenize text
    tokens = word_tokenize(text.lower())
    # Remove stopwords and numbers
    tokens = [word for word in tokens if word.isalpha() and word not in stopwords.words('english')]
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # print(tokens)
    return ' '.join(tokens)


# Run function and add to processed column
course_data_df['lemmatized_description'] = course_data_df['Robust'].apply(preprocess_text)

course_data_df[['Robust', 'lemmatized_description']].head()

,Robust,lemmatized_description
0,"BA (Geography) Environmental studies To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment",ba geography environmental study make student understand distribution utilization conservation natural resource crucial sustainable development involves studying earth renewable resource impact environment
1,"BA (Geography) Environmental studies Examining historical and current environmental movements provides insight into societal responses to environmental issues. Environmental management strategies help students understand how to balance development with conservation. Students will gain insights into waste disposal techniques, recycling processes, and the impact of improper waste management on the environment",ba geography environmental study examining historical current environmental movement provides insight societal response environmental issue environmental management strategy help student understand balance development conservation student gain insight waste disposal technique recycling process impact improper waste management environment
2,"BA (Geography) Environmental Science-I To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment",ba geography environmental make student understand distribution utilization conservation natural resource crucial sustainable development involves studying earth renewable resource impact environment
3,"BA (Geography) Environmental Science-II Managing waste is critical for preventing environmental degradation. Students need to understand waste generation, treatment, and disposal methods to contribute to sustainable waste management practices.Students will comprehend the causes and effects of pollution, as well as the regulatory frameworks and policies aimed at environmental conservation.",ba geography environmental managing waste critical preventing environmental degradation student need understand waste generation treatment disposal method contribute sustainable waste management comprehend cause effect pollution well regulatory framework policy aimed environmental conservation
4,"BA (Geography) Fundamentals of Geomorphology These geological processes shape the Earth's surface. Understanding endogenic (internal) and exogenic (external) processes helps in comprehending landforms, earthquakes,\nand erosion.",ba geography fundamental geomorphology geological process shape earth surface understanding endogenic internal exogenic external process help comprehending landforms earthquake erosion


In [49]:
print(lemmatizer.lemmatize('universal'))

universal


In [50]:
course_data_df.head()

,SrNo,Name Of the Program,Type of Course,Code,Need,Description of the need,Robust,lemmatized_description
0,1,BA (Geography),Environmental studies,UGEVS101,"Local, Regional, National and Global","To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment","BA (Geography) Environmental studies To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment",ba geography environmental study make student understand distribution utilization conservation natural resource crucial sustainable development involves studying earth renewable resource impact environment
1,2,BA (Geography),Environmental studies,UGEVS201,"Local, Regional, National and Global","Examining historical and current environmental movements provides insight into societal responses to environmental issues. Environmental management strategies help students understand how to balance development with conservation. Students will gain insights into waste disposal techniques, recycling processes, and the impact of improper waste management on the environment","BA (Geography) Environmental studies Examining historical and current environmental movements provides insight into societal responses to environmental issues. Environmental management strategies help students understand how to balance development with conservation. Students will gain insights into waste disposal techniques, recycling processes, and the impact of improper waste management on the environment",ba geography environmental study examining historical current environmental movement provides insight societal response environmental issue environmental management strategy help student understand balance development conservation student gain insight waste disposal technique recycling process impact improper waste management environment
2,3,BA (Geography),Environmental Science-I,UGESlOl,"Local, Regional, National and Global","To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment","BA (Geography) Environmental Science-I To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment",ba geography environmental make student understand distribution utilization conservation natural resource crucial sustainable development involves studying earth renewable resource impact environment
3,4,BA (Geography),Environmental Science-II,UGES201,"Local, Regional, National and Global","Managing waste is critical for preventing environmental degradation. Students need to understand waste generation, treatment, and disposal methods to contribute to sustainable waste management practices.Students will comprehend the causes and effects of pollution, as well as the regulatory frameworks and policies aimed at environmental conservation.","BA (Geography) Environmental Science-II Managing waste is critical for preventing environmental degradation. Students need to understand waste generation, treatment, and disposal methods to contribute to sustainable waste management practices.Students will comprehend the causes and effects of pollution, as well as the regulatory frameworks and policies aimed at environmental conservation.",ba geography environmental managing waste critical preventing environmental degradation student need understand waste generation treatment disposal method contribute sustainable waste management co

In [52]:
corpus = course_data_df['Robust']

tfidf_vectorizer = TfidfVectorizer()
count_vectorizer = CountVectorizer()

x = tfidf_vectorizer.fit_transform(corpus)
tfidf_vectorizer.get_feature_names_out()

array(['11', '1550', '16th', ..., 'your', 'zealand', 'zoology'],
      dtype=object)

In [53]:
x.shape

(879, 2602)

In [54]:
# Vectorizer 

corpus2 = count_vectorizer.fit_transform(corpus)
#print(corpus2)

print(count_vectorizer.get_feature_names_out())


['11' '1550' '16th' ... 'your' 'zealand' 'zoology']


In [55]:
cv_df = pd.DataFrame(corpus2.toarray(), columns=count_vectorizer.get_feature_names_out())

In [56]:
cv_df

,11,1550,16th,1832,18th,1980,19th,1cybersecurity,1potential,1pv4,...,wrapper,write,writing,wsns,years,yield,you,your,zealand,zoology
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
875,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
876,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
model = NearestNeighbors(n_neighbors=5, metric='cosine')

model.fit(x)

# This function will find and return recommended courses, n = # of results returned
def course_finder(description, n=5):
    # Use the preprocess function built above
    processed_description = preprocess_text(description)
    # Transform the input description into TF-IDF features - similar to corpus vectorizer above
    description_tfidf = tfidf_vectorizer.transform([processed_description])
    # Find KNN
    distances, indices = model.kneighbors(description_tfidf, n_neighbors=n)
    recommendations = course_data_df.iloc[indices[0]]
    return recommendations[['Code', 'Name Of the Program', 'Type of Course', 'Description of the need']]

# Example usage: Recommend courses based on a sample description
course_description = "I want to learn about natural language processing"
course_recommendation = course_finder(course_description)
course_recommendation

,Code,Name Of the Program,Type of Course,Description of the need
841,PGIT403,MSc (Information\nTechnology),Natural Language Processing,Natural language processing helps computers communicate with humans in their own language.
781,PGDS303,MSC (Data Science),Natural Language Processing,"NLP can be applied to enhance communication between local businesses and customers. Chatbots powered by NLP can provide instant customer support, process inquiries, and facilitate smoother interactions."
651,PGCHODSE404 A,MSc (Organic Chemistry),Natural Products and Heterocvclic Chemistry,Students are exposed to reaction mechanisms which are Carried out bv natural chemicals
842,PGIT4P3,MSc (Information Technology),Natural Language Processing Practical,"Technologies are critical for enterprises that handle a lot of unstructured text Sentiment analysis, chatbots, text extraction, text summarization, and speech recognition are\nsome real-life applications."
668,PGMB104- DSEC- 2,MSc (Microbiology),Food&Dairy Microbiology,Students will learn various aspects of microbiology involved in Food and Dairy industries and their processing.
